Setup

In [ ]:
# # Create results folder if it does not exist
# if not os.path.exists("../results"):
#     os.makedirs("../results")

# profit_comparison_results = pd.read_csv("../results/profit_comparison_0.005_3600_10000_1_2_100000000.csv", index_col=0)\

# Import libraries
import pandas as pd
import numpy as np
import csv
import os

import matplotlib.pyplot as plt
from matplotlib import rcParams
import statsmodels.api as sm
import glob
import scipy.stats
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from matplotlib import cm
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib as mpl
from io import StringIO

dpi = 300

P_VALUE_THRESHOLD = 0.05

of_colors = [
    "#fd7f6f",  # salmon
    "#7eb0d5",  # sky blue
    "#b2e061",  # lime green
    "#bd7ebe",  # lavender pink
    "#ffb55a",  # orange
    "#ffee65",  # lemon yellow
    "#beb9db",  # light lavender
    "#fdcce5",  # light pink
    "#8bd3c7",  # seafoam green
    "#ab82ff",  # lavender purple
    "#74c365",  # soft green
    "#808080",  # soft slate gray
]

colors_blues = cm.get_cmap('Blues')
colors_reds = cm.get_cmap('Reds')

colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#9467bd", "#7f7f7f"]

# Get tab20 colors in iterable format
tab20 = cm.get_cmap('tab20')
tab20 = [tab20.colors[i] for i in range(20)]

paired = cm.get_cmap('Paired')
paired = [paired.colors[i] for i in range(12)]

# # plt.style.use('default')
# plt.rcParams.update({
#     "text.usetex": True,  # Use LaTeX for rendering text
#     "font.family": "serif",  # Use serif font
#     "font.serif": ["Computer Modern"]
# })

# # Hide top and right spines
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
custom_palette = of_colors

FIG_SIZE = (6.4, 4)
FIG_SIZE_2 = (8, 6)
SAVE_FIG = True
FIG_FMT = 'pdf'
TRANSPARENT_PNG=True

Common Functions

In [ ]:
def dex_out_with_fee(x_in, L_in, L_out, fee):
    """
    Computes the output amount on a DEX swap, taking into account the fee.
    Works elementwise if L_in and L_out are numpy arrays.
    """
    effective_in = x_in * (1 - fee)
    return (L_out * effective_in) / (L_in + effective_in)


def round_trip_profit_dex_dex(
    delta_y, dex1_x, dex1_y, dex2_x, dex2_y, dex1_fee, dex2_fee
):
    """
    Computes the immediate DEX-DEX arbitrage profit (no bridging).
    """
    x_out = dex_out_with_fee(delta_y, dex1_y, dex1_x, dex1_fee)
    y_out = dex_out_with_fee(x_out, dex2_x, dex2_y, dex2_fee)
    return y_out - delta_y


def vectorized_round_trip_profit_bridge(
    delta_y, dex1_y, dex1_x, dex1_fee, dex2_x, dex2_y, dex2_fee, bridging_fee, p_sim
):
    """
    Computes the cross-chain bridging arbitrage profit for a vector of simulated DEX2 prices.
    All heavy computations are performed with numpy vectorized operations.
    """
    # Step 1: Swap Y->X on DEX1 (scalar since delta_y is constant)
    x_out = dex_out_with_fee(delta_y, dex1_y, dex1_x, dex1_fee)
    # Step 2: Apply the bridging fee
    x_after_bridge = x_out * (1 - bridging_fee)
    # Step 3: Rebalance DEX2 reserves based on simulated prices and swap X->Y on DEX2 vectorized
    K = dex2_x * dex2_y
    X2_arbed = np.sqrt(K / p_sim)
    Y2_arbed = np.sqrt(K * p_sim)
    y_out = dex_out_with_fee(x_after_bridge, X2_arbed, Y2_arbed, dex2_fee)
    return y_out - delta_y

def round_trip_profit_cex_dex(delta_y, dex1_x, dex1_y, dex1_fee, cex_price, cex_fee):
    """Immediate CEX-DEX arbitrage. Sell on CEX after DEX1 swap."""
    x_out = dex_out_with_fee(delta_y, dex1_y, dex1_x, dex1_fee)
    y_cex = x_out * cex_price * (1 - cex_fee)  # Sell on CEX
    return y_cex - delta_y

Bridge Durations on Profit Difference

In [ ]:
static_bridge_fee = 0.005
max_bridge_duration = 3600
simulation_count = 100
annual_sigma = 1
initial_price_factor = 2
M = 10_000_00

# DEX1 parameters
dex1_x, dex1_y = 1e6, 1e6
dex1_fee = 0.003
p_dex1 = dex1_y / dex1_x

# DEX2 parameters
dex2_x = 1e6
dex2_fee = 0.003

# Simulation setup
bridge_durations = np.linspace(0, max_bridge_duration, simulation_count)

# ETH-USD market parameters
mu_annual = 0.0
seconds_per_year = 365 * 24 * 3600
mu = mu_annual / seconds_per_year
sigma = annual_sigma / np.sqrt(seconds_per_year)

# Initial DEX2 price (DEX2 starts more expensive)
p2 = p_dex1 * initial_price_factor
dex2_y = dex2_x * p2

# 1) Calculate optimal DEX input based on the initial difference
# denominator = dex2_x + dex1_x * (1 - dex1_fee)
# y = (dex1_y * dex2_x) / denominator
# y_ = (dex2_y * dex1_x * (1 - dex1_fee)) / denominator
# opt_dex_input = ((np.sqrt(y * y_ * (1 - dex1_fee))) - y) / (1 - dex1_fee)
denominator = dex2_x + dex1_x
y = (dex1_y * dex2_x) / denominator
y_ = (dex2_y * dex1_x) / denominator
opt_dex_input = ((np.sqrt(y * y_ )) - y)
opt_dex_input = max(opt_dex_input, 0)

# 2) Compute the instant DEX-DEX arbitrage profit (without bridging)
max_profit_dex = round_trip_profit_dex_dex(
    opt_dex_input, dex1_x, dex1_y, dex2_x, dex2_y, dex1_fee, dex2_fee
)

profit_diffs_duration = []

# Loop over each bridge duration
for i, T_sec in enumerate(bridge_durations):
    dt = T_sec
    # Simulate final prices after T_sec for M simulations using a vectorized approach
    W_T = np.random.normal(0, np.sqrt(dt), M)
    p2_simulated = (dex2_y / dex2_x) * np.exp((mu - 0.5 * sigma**2) * dt + sigma * W_T)

    # Compute bridging profit for all simulated prices at once
    bridging_profits = vectorized_round_trip_profit_bridge(
        opt_dex_input,
        dex1_y,
        dex1_x,
        dex1_fee,
        dex2_x,
        dex2_y,
        dex2_fee,
        static_bridge_fee,
        p2_simulated,
    )

    bridge_dex_expected_profit = np.mean(bridging_profits)
    profit_diffs_duration.append(max_profit_dex - bridge_dex_expected_profit)

In [ ]:
# Plot the results
window_size = 50  # Adjust as needed
profit_diffs_series = pd.Series(profit_diffs_duration)
moving_avg = profit_diffs_series.rolling(
    window=window_size, center=True, min_periods=1
).mean()

# Plot the profit differences and the moving average versus bridge duration
fig, ax = plt.subplots(figsize=(6.4, 4), dpi=300)
ax.plot(
    bridge_durations,
    profit_diffs_duration,
    linestyle="-",
    label="Profit Difference",
)
ax.plot(bridge_durations, moving_avg, linestyle="--", label="Moving Average")
ax.set_xlabel("Bridging Duration (s)", fontsize = 12)
ax.set_ylabel("Profit Difference", fontsize = 12)
ax.tick_params(axis='both', labelsize=12)

# ax.set_title(
#     f"Profit Difference vs. Bridge Duration"
# )
ax.legend()
ax.grid(True, linestyle='--', linewidth=0.5)

# Limit x axis to 0-max_bridge_duration
ax.set_xlim(0, max_bridge_duration)

plt.tight_layout()
plt.savefig(
    f"../results/profit_diffs_vs_duration_{static_bridge_fee}_{max_bridge_duration}_{simulation_count}_{annual_sigma}_{initial_price_factor}.pdf",
    bbox_inches="tight",
    facecolor="auto",
    edgecolor="auto",
)

# plt.show()

# Save the results to a CSV file
results_df = pd.DataFrame(
    {
        "Bridge Duration": bridge_durations,
        "Profit Difference": profit_diffs_duration,
        "Moving Average": moving_avg,
    }
)
results_df.to_csv(
    f"../results/profit_diffs_vs_duration_{static_bridge_fee}_{max_bridge_duration}_{simulation_count}_{annual_sigma}_{initial_price_factor}.csv",
    index=False,
)

Initial Price Difference on Profit Difference

In [ ]:
price_sim_count = 10000
max_price_diff = 2
annual_sigma = 1
bridging_delay = 3600
M = 100_000_000

# DEX1 parameters
dex1_x, dex1_y = 1e6, 1e6
dex1_fee = 0.003
p_dex1 = dex1_y / dex1_x

# DEX2 parameters
dex2_x = 1e6
dex2_fee = 0.003

# CEX parameters
cex_fee = 0.001  # 0.1% fee

# Bridging parameters
bridging_fee = 0.005  # 0.5% bridging fee

# GBM parameters
mu_annual = 0.0
seconds_per_year = 365 * 24 * 3600
mu = mu_annual / seconds_per_year
sigma = annual_sigma / np.sqrt(seconds_per_year)

# Define a range of final prices p2, for immediate DEX2 or CEX
p2_values = np.linspace(p_dex1, p_dex1 * max_price_diff, price_sim_count)
price_diff = p2_values - p_dex1

# Arrays for storing the max profits (each point in p2_values)
max_profits_dex_dex = []
max_profits_cex_dex = []
max_profits_bridge_dex_mc = []

###############################################################################
# Outer loop: for each immediate final price p2 of the second DEX
###############################################################################
for p2 in p2_values:
    # Re-compute dex2_y for that p2
    dex2_y = dex2_x * p2

    # 1) Instant DEX-DEX: find best input
    denominator = dex2_x + dex1_x * (1 - dex1_fee)
    y = (dex1_y * dex2_x) / denominator
    y_ = (dex2_y * dex1_x * (1 - dex1_fee)) / denominator
    opt_dex_input = ((np.sqrt(y * y_ * (1 - dex1_fee))) - y) / (1 - dex1_fee)
    opt_dex_input = max(opt_dex_input, 0)
    # 2) Compute the instant DEX-DEX arbitrage profit (without bridging)
    max_profit_dex = round_trip_profit_dex_dex(
        opt_dex_input, dex1_x, dex1_y, dex2_x, dex2_y, dex1_fee, dex2_fee
    )
    max_profits_dex_dex.append(max_profit_dex)

    # 3) Instant CEX-DEX: find best input
    opt_cex_input = (np.sqrt(dex1_x*dex1_y * p2 * (1 - dex1_fee)*(1-cex_fee)) - dex1_y)/(1 - dex1_fee)
    cex_input = max(opt_cex_input, 0)
    # 4) Compute the instant CEX-DEX arbitrage profit
    max_profit_cex = round_trip_profit_cex_dex(cex_input, dex1_x, dex1_y, dex1_fee, p2, cex_fee)
    max_profits_cex_dex.append(max_profit_cex)

    # 5) Cross-chain bridging arbitrage: Monte Carlo simulation
    dt = bridging_delay
    # Simulate final prices after T_sec for M simulations using a vectorized approach
    W_T = np.random.normal(0, np.sqrt(dt), M)
    p2_simulated = (dex2_y / dex2_x) * np.exp(
        (mu - 0.5 * sigma**2) * dt + sigma * W_T
    )
    # Compute bridging profit for all simulated prices at once
    bridging_profits = vectorized_round_trip_profit_bridge(
        opt_dex_input,
        dex1_y,
        dex1_x,
        dex1_fee,
        dex2_x,
        dex2_y,
        dex2_fee,
        bridging_fee,
        p2_simulated,
    )
    bridge_dex_expected_profit = np.mean(bridging_profits)
    max_profits_bridge_dex_mc.append(bridge_dex_expected_profit)

# Convert results to numpy arrays
max_profits_dex_dex = np.array(max_profits_dex_dex)
max_profits_cex_dex = np.array(max_profits_cex_dex)
max_profits_bridge_dex_mc = np.array(max_profits_bridge_dex_mc)

prof_diff_dex_dex_to_bridge_dex = max_profits_dex_dex - max_profits_bridge_dex_mc
prof_diff_cex_dex_to_bridge_dex = max_profits_cex_dex - max_profits_bridge_dex_mc
prof_diff_cex_dex_to_dex_dex = max_profits_cex_dex - max_profits_dex_dex

In [ ]:
# Save the results to a CSV file
df = pd.DataFrame(
    {
        "p2": p2_values,
        "price_diff": price_diff,
        "max_profits_dex_dex": max_profits_dex_dex,
        "max_profits_cex_dex": max_profits_cex_dex,
        "max_profits_bridge_dex_mc": max_profits_bridge_dex_mc,
        "prof_diff_dex_dex_to_bridge_dex": prof_diff_dex_dex_to_bridge_dex,
        "prof_diff_cex_dex_to_bridge_dex": prof_diff_cex_dex_to_bridge_dex,
        "prof_diff_cex_dex_to_dex_dex": prof_diff_cex_dex_to_dex_dex,
    }
)
df.to_csv(f"../results/profit_comparison_{bridging_fee}_{bridging_delay}_{price_sim_count}_{annual_sigma}_{max_price_diff}_{M}.csv", index=False)

In [ ]:
if profit_comparison_results is None:
    profit_comparison_results = df

max_price_diff = profit_comparison_results["price_diff"].max()

fig, ax = plt.subplots(figsize=(6.4, 4), dpi=300)

# Main plots (Primary Y-axis) with distinct line styles
line1, = ax.plot(profit_comparison_results["price_diff"], profit_comparison_results["max_profits_dex_dex"], label="Instant DEX-DEX", lw=2)
line2, = ax.plot(profit_comparison_results["price_diff"], profit_comparison_results["max_profits_bridge_dex_mc"], label="Bridge DEX-DEX", lw=2, linestyle='--')
line3, = ax.plot(profit_comparison_results["price_diff"], profit_comparison_results["max_profits_cex_dex"], label="CEX-DEX", lw=2)

ax.set_xlabel("Instant Price Discrepancy", fontsize=12)
ax.set_ylabel("Maximum Arbitrage Profit", fontsize=12)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True, linestyle='--', linewidth=0.5)

# Make y-axis scientific
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# Limit x axis to 0 - max_price_diff
ax.set_xlim(0, profit_comparison_results["price_diff"].max())

# Secondary plot (Secondary Y-axis)
ax2 = ax.twinx()
line4, = ax2.plot(
    profit_comparison_results["price_diff"],
    profit_comparison_results["prof_diff_dex_dex_to_bridge_dex"],
    linestyle="-.",
    label="Instant vs. Bridge",
    color="gray",
    alpha=0.6,
    linewidth=2,
)

ax2.set_ylabel("Profit Difference", color="gray", fontsize=12)
ax2.tick_params(axis='y', labelcolor="gray", labelsize=12)

lines = [line1, line2, line3, line4]
labels = [l.get_label() for l in lines]
ax.legend(lines, labels, loc='upper left')

plt.tight_layout()

# Save as PDF
plt.savefig(f"../results/profit_vs_price_diff_{max_price_diff}.pdf", 
            bbox_inches='tight', facecolor='auto', edgecolor='auto')
# plt.show()


## Final Experiments

Profit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
num_points = 10**3
p = np.linspace(1, 2, num_points)  # price ratio (p = p2/p1)
r_a = 10**7
profits = (np.sqrt(p) - 1)**2 * r_a

# Plot
plt.figure(dpi=dpi, figsize=FIG_SIZE)
plt.plot(p, profits,  color= colors[0], lw=2)
plt.xlabel(r'Relative Price ($p=Q/P)$', fontsize=16)
plt.ylabel('Profit', fontsize=16)
# plt.title(r'$\Delta(\lambda)$ as a function of $\lambda$')
plt.grid(True, linestyle='--', linewidth=0.5)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend()
# Limit x axis to 1 - 2
# plt.xlim(1, 2)
plt.tight_layout()
# plt.show()

plt.tick_params(labelsize=16)

# Save the plot
plt.savefig(f"arb_profit.pdf", bbox_inches='tight', facecolor='auto', edgecolor='auto')


Cost of bridge

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p = 2  # price ratio (p = p2/p1)
mu = -1/16
num_points = 10**3
deltas = np.linspace(0, 10, num_points)  # delta values
cost_of_bridge = p*(1-np.exp(mu*deltas)) - 2*np.sqrt(p)*(1-np.exp(mu*deltas/2))
cost_of_bridge_2 = (np.sqrt(p) - 1)**2 - (np.sqrt(p)*np.exp(mu*deltas/2) - 1)**2

# Plot
plt.figure(dpi=dpi, figsize=FIG_SIZE)
plt.plot(deltas, cost_of_bridge,  color= colors[0], lw=2)
plt.xlabel(r'Bridging Time ($\Delta$)', fontsize=16)
plt.ylabel(r'Cost of Non-Instant Bridging ($C^{BR}$)', fontsize=14)
# plt.title(r'$\Delta(\lambda)$ as a function of $\lambda$')
plt.grid(True, linestyle='--', linewidth=0.5)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend()
# Limit x axis to 1 - 2
# plt.xlim(0, 10)
plt.tight_layout()
# plt.show()

plt.tick_params(labelsize=16)

# Save the plot
plt.savefig(f"non_instant_bridge_cost.pdf", bbox_inches='tight', facecolor='auto', edgecolor='auto')


Cost of bridge vs mu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p = 4  # price ratio (p = p2/p1)
mus = np.linspace(-1, 0, 1000)
num_points = 10**3
delta = 2  # delta values
cost_of_bridge_2 = (np.sqrt(p) - 1)**2 - (np.sqrt(p)*np.exp(mus*delta/2) - 1)**2

# Plot
plt.figure(dpi=dpi, figsize=FIG_SIZE)
plt.plot(mus, cost_of_bridge_2,  color= colors[0], lw=2)
plt.xlabel(r'Mu', fontsize=16)
plt.ylabel(r'Cost of Non-Instant Bridging ($C^{BR}$)', fontsize=14)
# plt.title(r'$\Delta(\lambda)$ as a function of $\lambda$')
plt.grid(True, linestyle='--', linewidth=0.5)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend()
# Limit x axis to 1 - 2
# plt.xlim(0, 10)
plt.tight_layout()
# plt.show()

plt.tick_params(labelsize=16)

delta vs lambda

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p = 2  # price ratio (p = p2/p1)
mu = -1/16
sigma = 1/2
num_points = 10**3

deltas = np.linspace(0, 1, num_points)  # delta values
cost_of_bridge = (np.sqrt(p) - 1)**2 - (np.sqrt(p)*np.exp(mu*deltas/2) - 1)**2
lambdas_constant_liq = ((-mu) / cost_of_bridge) * (1 - np.sqrt(1/p))

# lambdas_constant_liq = ((sigma**2 - mu) / cost_of_bridge) * (1 - np.sqrt(1/p))
# rho = mu/2 - (sigma**2)/8
# lambdas_no_flow = ((-2*rho) / cost_of_bridge) * (1 - np.sqrt(1/p))

# Plot
plt.figure(dpi=dpi, figsize=FIG_SIZE)
plt.plot(deltas, lambdas_constant_liq,  color= colors[0], lw=2)
# plt.plot(deltas, lambdas_no_flow,  color= colors[1], lw=2, linestyle='--', label ='Constant Liquidity')
plt.xlabel(r'Bridging Time ($\Delta$)', fontsize=16)
plt.ylabel(r'Arrival Rate ($\lambda$)', fontsize=16)
# plt.title(r'$\Delta(\lambda)$ as a function of $\lambda$')
plt.grid(True, linestyle='--', linewidth=0.5)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend(fontsize=14)
# Limit x axis to 1 - 2
# plt.xlim(0, 1)
# Make y axis logarithmic
plt.yscale('log')
plt.tight_layout()
# plt.show()

plt.tick_params(labelsize=16)

# Save the plot
plt.savefig(f"delta_vs_lambda.pdf", bbox_inches='tight', facecolor='auto', edgecolor='auto')


lambda vs delta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p = 2  # price ratio (p = p2/p1)
mu = -1/16
# sigma = 1/2
num_points = 10**3

lambdas = np.linspace(0, 1, num_points)  # delta values
# rho = mu/2 - (sigma**2)/8

M = (np.sqrt(p) - 1)**2 + (mu/(lambdas))*(1 - np.sqrt(1/p))

# term_inside_sqrt_constant_liq = 1 + p + 2 * np.sqrt(p) + (mu)/(lambdas*(1 - np.sqrt(1/p)))

# Drop <0 entries from term_inside_sqrt_constant_liq
# term_inside_sqrt_constant_liq = np.where(term_inside_sqrt_constant_liq < 0, np.nan, term_inside_sqrt_constant_liq)

# term_inside_sqrt_constant_liq = 1 + p + 2 * np.sqrt(p) - ((sigma**2) - mu)/(lambdas*(1 - np.sqrt(1/p)))
# term_inside_sqrt = 1 + (mu * (p + np.sqrt(p))) / (lambda_ * (lambda_ + mu)) + p - 2 * np.sqrt(p)
inner_log_argument_constant_liq =  np.sqrt(p) / (1 + np.sqrt(M))
delta_vals_constant_liq = -(2 / mu) * np.log(inner_log_argument_constant_liq)

# # Smallest lambda for which delta_vals_constant_liq > 0
# smallest_positive_lambda = lambdas[delta_vals_constant_liq > 0].min()

# Remove nan entries from delta_vals_constant_liq 

# term_inside_sqrt_no_flow = 1 + p + 2 * np.sqrt(p) - (-2*rho)/(lambdas*(1 - np.sqrt(1/p)))
# # term_inside_sqrt = 1 + (mu * (p + np.sqrt(p))) / (lambda_ * (lambda_ + mu)) + p - 2 * np.sqrt(p)
# inner_log_argument_no_flow = (-1 + np.sqrt(term_inside_sqrt_no_flow)) / np.sqrt(p)
# delta_vals_constant_no_flow = (2 / mu) * np.log(inner_log_argument_no_flow)

# Plot
plt.figure(dpi=dpi, figsize=FIG_SIZE)
plt.plot(lambdas, delta_vals_constant_liq,  color= colors[0], lw=2)
# plt.plot(lambdas, delta_vals_constant_no_flow,  color= colors[1], lw=2, linestyle='--', label ='Constant Liquidity')
plt.ylabel(r'Bridging Time ($\Delta$)', fontsize=16)
plt.xlabel(r'Arrival Rate ($\lambda$)', fontsize=16)
# plt.title(r'$\Delta(\lambda)$ as a function of $\lambda$')
plt.grid(True, linestyle='--', linewidth=0.5)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend(fontsize=14)
# Limit x axis to 1 - 2
# smallest_positive_lambda = delta_vals_constant_liq[delta_vals_constant_liq > 0].min()
# plt.xlim(smallest_positive_lambda, 1)
# plt.xlim(0, 1)
# Make y axis logarithmic
plt.yscale('log')
plt.tight_layout()
# plt.show()

plt.tick_params(labelsize=16)

# Save the plot
plt.savefig(f"lambda_vs_delta.pdf", bbox_inches='tight', facecolor='auto', edgecolor='auto')


mu threshold

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p = 2  # price ratio (p = p2/p1)
Lambda = 1
Delta = 1 # Set to 1
num_points = 10**3
mus = np.linspace(-1, 0, num_points)  # delta values

cost_of_bridge = (np.sqrt(p) - 1)**2 - (np.sqrt(p)*np.exp(mus*Delta/2) - 1)**2
cost_of_constant_liq = - (mus/Lambda) * (1 - np.sqrt(1/p)) 
# cost_of_no_flow = - (mu_hat_no_flow/Lambda) * (1 - np.sqrt(1/p))

cost_diff_constant_liq = cost_of_constant_liq - cost_of_bridge
# cost_diff_no_flow = cost_of_no_flow - cost_of_bridge

mus = mus * 100  # Convert to percentage

# Plot
plt.figure(dpi=dpi, figsize=FIG_SIZE)

plt.plot(mus, cost_diff_constant_liq,  color= colors[0], lw=2)
# plt.plot(sigmas, cost_diff_no_flow,  color= colors[1], lw=2, linestyle='--', label ='Constant Liquidity')

# Fill areas for cost_of_constant_liq
plt.fill_between(mus, cost_diff_constant_liq, 0,
                where=cost_diff_constant_liq > 0,
                color='lightgreen', alpha=0.25)
plt.fill_between(mus, cost_diff_constant_liq, 0,
                where=cost_diff_constant_liq < 0,
                color='lightcoral', alpha=0.25)

# # Fill areas for cost_diff_no_flow
# plt.fill_between(sigmas, cost_diff_no_flow, 0,
#                 where=cost_diff_no_flow > 0,
#                 color='lightgreen', alpha=0.25)
# plt.fill_between(sigmas, cost_diff_no_flow, 0,
#                 where=cost_diff_no_flow < 0,
#                 color='lightcoral', alpha=0.25)

plt.ylabel('Cost Difference (Inventory - Bridge)', fontsize=13.5)
plt.xlabel(r'% Drift ($\mu$)', fontsize=16)
# plt.title(r'$\Delta(\lambda)$ as a function of $\lambda$')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.axhline(0, color='grey', lw=0.8)
# plt.legend(fontsize=16)

# Limit x axis to 1 - 2
# plt.xlim(0, 100)
plt.tight_layout()
# plt.show()

plt.tick_params(labelsize=16)

# Save the plot
plt.savefig(f"volatility_vs_cost_diff.pdf", bbox_inches='tight', facecolor='auto', edgecolor='auto')

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Parameters
# p = 2  # price ratio (p = p2/p1)
# Lambda = 2
# Delta = 1
# num_points = 10**6
# sigmas = np.linspace(0, 1, num_points)  # delta values
# fixed_mu = -1/16

# mu_hat_constant_liq = fixed_mu - (sigmas**2)
# mu_hat_no_flow = fixed_mu - (sigmas**2)/4

# cost_of_bridge = (np.sqrt(p) - 1)**2 - (np.sqrt(p)*np.exp(fixed_mu*Delta/2) - 1)**2
# cost_of_constant_liq = - (mu_hat_constant_liq/Lambda) * (1 - np.sqrt(1/p)) 
# cost_of_no_flow = - (mu_hat_no_flow/Lambda) * (1 - np.sqrt(1/p))

# cost_diff_constant_liq = cost_of_constant_liq - cost_of_bridge
# cost_diff_no_flow = cost_of_no_flow - cost_of_bridge

# sigmas = sigmas * 100  # Convert to percentage

# # Plot
# plt.figure(dpi=dpi, figsize=FIG_SIZE)

# plt.plot(sigmas, cost_diff_constant_liq,  color= colors[0], lw=2, label ='Constant Value')
# plt.plot(sigmas, cost_diff_no_flow,  color= colors[1], lw=2, linestyle='--', label ='Constant Liquidity')

# # Fill areas for cost_of_constant_liq
# plt.fill_between(sigmas, cost_diff_constant_liq, 0,
#                 where=cost_diff_constant_liq > 0,
#                 color='lightgreen', alpha=0.25)
# plt.fill_between(sigmas, cost_diff_constant_liq, 0,
#                 where=cost_diff_constant_liq < 0,
#                 color='lightcoral', alpha=0.25)

# # Fill areas for cost_diff_no_flow
# plt.fill_between(sigmas, cost_diff_no_flow, 0,
#                 where=cost_diff_no_flow > 0,
#                 color='lightgreen', alpha=0.25)
# plt.fill_between(sigmas, cost_diff_no_flow, 0,
#                 where=cost_diff_no_flow < 0,
#                 color='lightcoral', alpha=0.25)

# plt.ylabel('Cost Difference (Inventory - Bridge)', fontsize=13.5)
# plt.xlabel(r'% Volatility ($\sigma$)', fontsize=14)
# # plt.title(r'$\Delta(\lambda)$ as a function of $\lambda$')
# plt.grid(True, linestyle='--', linewidth=0.5)
# plt.axhline(0, color='grey', lw=0.8)
# plt.legend(fontsize=12)

# # Limit x axis to 1 - 2
# plt.xlim(0, 100)
# plt.tight_layout()
# # plt.show()

# plt.tick_params(labelsize=14)

# # Save the plot
# plt.savefig(f"volatility_vs_cost_diff.pdf", bbox_inches='tight', facecolor='auto', edgecolor='auto')

## Others

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p = 2  # example value
mu = -0.8  # example negative rho, adjust as needed

# Lambda values (avoid zero to prevent division by zero)
lambda_vals = np.linspace(1, 20, 500)

# Delta as a function of lambda
def delta(lambda_):
    with np.errstate(divide='ignore', invalid='ignore'):
        term_inside_sqrt = 1 + (1 + (mu/(lambda_ * (lambda_ + mu))))*p - (2 - (mu/(lambda_ * (lambda_ + mu))))*np.sqrt(p)
        # term_inside_sqrt = 1 + (mu * (p + np.sqrt(p))) / (lambda_ * (lambda_ + mu)) + p - 2 * np.sqrt(p)
        inner_log_argument = (1 + np.sqrt(term_inside_sqrt)) / np.sqrt(p)
        delta_val = (2 / mu) * np.log(inner_log_argument)
        return np.real_if_close(delta_val)


# Compute delta values
delta_vals = delta(lambda_vals)

# Plot
plt.figure(dpi=dpi, figsize=FIG_SIZE)
plt.plot(lambda_vals, delta_vals, label=r'$\Delta(\lambda)$', color=colors[0], lw=2.5)
plt.xlabel(r'$\lambda$ (Arrival Rate)', fontsize=14)
plt.ylabel(r'$\Delta$ (Bridging Time)', fontsize=14)
# plt.title(r'$\Delta(\lambda)$ as a function of $\lambda$')
# plt.grid(True)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend()
plt.tight_layout()
# plt.show()

plt.tick_params(labelsize=14)

# Save the plot
plt.savefig(f"delta_lambda_plot.pdf", bbox_inches='tight', facecolor='auto', edgecolor='auto')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Fixed parameter
p = 2  # Inventory value

# === Plot 1: lambda as a function of rho (for fixed Delta) ===
Delta_fixed = 1.0  # Fixed bridging time

def lambda_from_rho(rho):
    lambda_vals = np.full_like(rho, np.nan)
    for i, r in enumerate(rho):
        if r < 0:
            # Ensure denominator is valid
            denom = 2 * (1 - np.exp(r * Delta_fixed / 2)) * np.sqrt(p) - (1 - np.exp(r * Delta_fixed)) * p
            if denom != 0:
                inside_sqrt = 4 * r * (p + np.sqrt(p)) / denom + r**2
                if inside_sqrt > 0:
                    lam = 0.5 * (r + np.sqrt(inside_sqrt))
                    if lam > 0 and r + lam > 0:  # Ensure lambda > 0 and lambda + rho > 0
                        lambda_vals[i] = lam
    return lambda_vals

rho_vals = np.linspace(-2, -0.01, 500)
lambda_rho_vals = lambda_from_rho(rho_vals)

# plt.figure(figsize=(8, 5))
# plt.plot(rho_vals, lambda_rho_vals, label=fr'$\lambda(\rho)$ for $\Delta={Delta_fixed}$')
# plt.xlabel(r'$\rho$')
# plt.ylabel(r'$\lambda$')
# plt.title(r'$\lambda$ as a function of $\rho$ (fixed $\Delta$)')
# plt.grid(True)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend()
# plt.tight_layout()
# plt.show()

# === Plot 2: lambda as a function of Delta (for fixed rho) ===
mu_fixed = -1/16  # Negative rho
sigma = 0.5

def lambda_from_delta(Delta):
    lambda_vals = np.full_like(Delta, np.nan) 
    for i, d in enumerate(Delta):
        rho = mu_fixed/2 - sigma**2/8
        C = (1 - np.exp(mu_fixed * d)) * p - 2*(1 - np.exp(rho * d/2)) * np.sqrt(p)
        inside_sqrt = mu_fixed**2 - (4*mu_fixed*(p - np.sqrt(p))/C)
        # denom = 2 * (1 - np.exp(mu_fixed * d / 2)) * np.sqrt(p) - (1 - np.exp(mu_fixed * d)) * p
        # if denom != 0:
        # inside_sqrt = 4 * mu_fixed * (p + np.sqrt(p)) / denom + mu_fixed**2
        if inside_sqrt > 0:
            lam = 0.5 * (-mu_fixed + np.sqrt(inside_sqrt))
            if lam > 0 and lam + mu_fixed > 0:
                lambda_vals[i] = lam
    return lambda_vals

Delta_vals = np.linspace(0, 1, 500)
lambda_delta_vals = lambda_from_delta(Delta_vals)

plt.figure(dpi=dpi, figsize=FIG_SIZE)
plt.plot(Delta_vals, lambda_delta_vals, label=fr'$\lambda(\Delta)$ for $\rho={mu_fixed:.4f}$', color=colors[0], lw=2.5) 
plt.xlabel(r'$\Delta$ (Bridging Time)', fontsize=14)
plt.ylabel(r'$\lambda$ (Arrival Rate)', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
# Liit x axis to 0-1
# plt.title(r'$\lambda$ as a function of $\Delta$ (fixed $\rho$)')
# plt.grid(True)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend()
plt.tight_layout()
plt.savefig(f"lambda_vs_rho_delta.pdf", bbox_inches='tight', facecolor='auto', edgecolor='auto')

# plt.show()

# # === Plot 3: Inventory profit as a function of lambda ===
# reserve = 10**9
#   # example reserve value
# rho_fixed = -1  # Negative rho
# p = 100
# def get_inventory_profit(lambda_):
#     profit = (p - 2 * np.sqrt(p) + 1) * reserve - (- rho_fixed / (lambda_ * (lambda_ + rho_fixed))) * (p - np.sqrt(p)) * reserve
#     return profit
# def get_inventory_profit_v2(lambda_):
#     return (1.0 + rho_fixed / (lambda_ * (lambda_ + rho_fixed))) * p \
#            - (2.0 - rho_fixed / (lambda_ * (lambda_ + rho_fixed))) * np.sqrt(p)

# lambda_vals = np.linspace(1.1, 10, 500)
# inventory_profit_vals = get_inventory_profit_v2(lambda_vals)

# plt.figure(figsize=(8, 5))
# plt.plot(lambda_vals, inventory_profit_vals, label=fr'Inventory Profit for $\rho={rho_fixed:.4f}$')
# plt.xlabel(r'$\lambda$')
# plt.ylabel(r'Inventory Profit')
# plt.title(r'Inventory Profit as a function of $\lambda$')
# plt.grid(True)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
# plt.legend()
# plt.tight_layout()
# plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# 1) Define Parameters
# -----------------------------
p = 100.0            # example
lambda_ = 2.0        # arrival rate
rho = -0.5           # negative drift for decay
Delta_min = 0.0
Delta_max = 100.0
num_points = 200

# -----------------------------
# 2) Define Payoffs as Functions of Delta
# -----------------------------
def bridge_payoff(Delta):
    return p * np.exp(rho * Delta) - 2.0 * np.sqrt(p) * np.exp(rho * Delta / 2.0)

# Inventory payoff (constant in Delta)
inventory_payoff = (1.0 + rho / (lambda_ * (lambda_ + rho))) * p \
                   - (2.0 - rho / (lambda_ * (lambda_ + rho))) * np.sqrt(p)

# -----------------------------
# 3) Generate a Range of Deltas
# -----------------------------
Delta_vals = np.linspace(Delta_min, Delta_max, num_points)

# Compute bridging profits over that range
bridge_vals = bridge_payoff(Delta_vals)

# -----------------------------
# 4) Plot the Results
# -----------------------------
plt.figure(figsize=(8,5))

plt.plot(Delta_vals, bridge_vals, label="Bridge Payoff", linewidth=2)
plt.axhline(inventory_payoff, color='red', linestyle='--', 
            label="Inventory Payoff (constant)")

plt.title("Bridge Payoff vs. Bridging Time Δ")
plt.xlabel("Δ (bridging time)")
plt.ylabel("Payoff")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # -----------------------------
# # 1) Define Parameters
# # -----------------------------
# p = 100.0            # example
# lambda_ = 3          # arrival rate
# Delta = 5.0
# num_points = 2000

# # -----------------------------
# # 2) Define Payoffs as Functions of Rho
# # -----------------------------
# def bridge_payoff(rho_):
#     return p * np.exp(rho_ * Delta) - 2.0 * np.sqrt(p) * np.exp(rho_ * Delta / 2.0)

# def inventory_payoff(rho_):
#     return (1.0 + rho_ / (lambda_ * (lambda_ + rho_))) * p \
#            - (2.0 - rho_ / (lambda_ * (lambda_ + rho_))) * np.sqrt(p)

# # -----------------------------
# # 3) Generate a Range of Rhos
# # -----------------------------
# rho_vals = np.linspace(-2.1, -0.001, num_points)
# bridge_vals = bridge_payoff(rho_vals)
# inventory_vals = inventory_payoff(rho_vals)
# profit_diff = inventory_vals - bridge_vals

# # Find the threshold rho where profit_diff crosses zero
# threshold_idx = np.where(np.diff(np.sign(profit_diff)))[0][0]
# rho_threshold = rho_vals[threshold_idx]

# # -----------------------------
# # 4) Plot the Results
# # -----------------------------
# plt.figure(figsize=(8,5))
# plt.plot(rho_vals, profit_diff, label="Profit Difference (Inventory - Bridge)", linewidth=2)
# plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
# plt.axvline(rho_threshold, color='red', linestyle='--', label=f"Threshold $\\rho$ = {rho_threshold:.4f}")

# plt.title("Profit Difference vs. Rho")
# plt.xlabel("Rho")
# plt.ylabel("Profit Difference")
# plt.grid(True)
# plt.legend()
# plt.tight_layout()
# plt.show()

import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# 1) Define Parameters
# -----------------------------
p = 2.0            # example
Lambda = 2          # arrival rate
Delta = 10
num_points = 10**6
sigma = 0.5

# -----------------------------
# 2) Define Payoffs as Functions of Rho
# -----------------------------
def bridge_cost(mu_):
    rho = mu_/2 - sigma**2/8
    return (1 - np.exp(mu_ * Delta)) * p - 2*(1 - np.exp(rho * Delta/2)) * np.sqrt(p)

def inventory_cost_case_1(mu_):
    if Lambda > 0 and Lambda + mu_ > 0:
        return (- mu_ / (Lambda * (Lambda + mu_))) * (p - np.sqrt(p))

def inventory_cost_case_2(mu_):
    rho = mu_/2 - sigma**2/8
    if rho > -2*Lambda:
        return (- rho / (Lambda * (Lambda + rho/2))) * (p - np.sqrt(p))
# -----------------------------
# 3) Generate a Range of Rhos
# -----------------------------
mu_vals = np.linspace(-4, 0, num_points)
bridge_costs = bridge_cost(mu_vals)
inventory_costs_1 = inventory_cost_case_1(mu_vals)
inventory_costs_2 = inventory_cost_case_2(mu_vals)
cost_diff_1 = inventory_costs_1 - bridge_costs
cost_diff_2 = inventory_costs_2 - bridge_costs

# # Find the threshold rho where profit_diff crosses zero
# threshold_idx = np.where(np.diff(np.sign(cost_diff)))[0][0]
# rho_threshold = rho_vals[threshold_idx]

# -----------------------------
# 4) Plot the Results
# -----------------------------
plt.figure(figsize=(8,5))
plt.plot(rho_vals, cost_diff, label="Cost Difference (Inventory - Bridge)", lw=2.5, color=colors[0])
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# # Highlight regions
# plt.fill_between(rho_vals, cost_diff, where=rho_vals < rho_threshold, color='lightcoral', alpha=0.3, label="Bridge Favored")
# plt.fill_between(rho_vals, cost_diff, where=rho_vals >= rho_threshold, color='lightgreen', alpha=0.3, label="Inventory Favored")

# plt.title("Cost Difference vs. Mu")
plt.ylabel("Cost Difference (Inventory - Bridge)", fontsize=14)
plt.xlabel(r'$\mu$ (Drift)', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
# plt.grid(True)
# plt.legend()
plt.tight_layout()
# Limit x axis to -2.99-0
plt.xlim(-4, 0)
# plt.ylim(-2, 2)
# plt.show()

# Save the plot
plt.savefig("profit_difference_vs_rho.pdf", 
            bbox_inches='tight', facecolor='auto', edgecolor='auto')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
p = 2.0
Lambda = 1
Delta = 1
num_points = 10**6
sigma = 0.5
min_mu = -1/4
max_mu = 0

# Payoff functions
def bridge_cost(mu_):
    rho = mu_/2 - sigma**2/8
    return (1 - np.exp(mu_ * Delta)) * p - 2*(1 - np.exp(rho * Delta/2)) * np.sqrt(p)

def inventory_cost_case_1(mu_):
    valid = (Lambda > 0) & (Lambda + mu_ > 0)
    result = np.full_like(mu_, np.nan)
    result[valid] = (- mu_[valid] / (Lambda * (Lambda + mu_[valid]))) * (p - np.sqrt(p))
    return result

def inventory_cost_case_2(mu_):
    rho = mu_/2 - sigma**2/8
    valid = (rho > -2 * Lambda)
    result = np.full_like(mu_, np.nan)
    result[valid] = (- rho[valid] / (Lambda * (Lambda + rho[valid]/2))) * (p - np.sqrt(p))
    return result

# Grid
mu_vals = np.linspace(min_mu, max_mu - 0.001, num_points)
bridge_vals = bridge_cost(mu_vals)
inv_vals_1 = inventory_cost_case_1(mu_vals)
inv_vals_2 = inventory_cost_case_2(mu_vals)

cost_diff_1 = inv_vals_1 - bridge_vals
cost_diff_2 = inv_vals_2 - bridge_vals

# Plot
plt.figure(dpi=dpi, figsize=FIG_SIZE)

# Lines
plt.plot(mu_vals, cost_diff_1, label="Inventory Case 1 (with inflow)", lw=2, color=colors[0])
plt.plot(mu_vals, cost_diff_2, label="Inventory Case 2 (no inflow)", lw=2, linestyle='--', color=colors[1])

# Fill areas for cost_diff_1
plt.fill_between(mu_vals, cost_diff_1, 0, where=cost_diff_1 > 0, interpolate=True, color='lightgreen', alpha=0.3)
plt.fill_between(mu_vals, cost_diff_1, 0, where=cost_diff_1 < 0, interpolate=True, color='lightcoral', alpha=0.3)

# Fill areas for cost_diff_2
plt.fill_between(mu_vals, cost_diff_2, 0, where=cost_diff_2 > 0, interpolate=True, color='lightgreen', alpha=0.3)
plt.fill_between(mu_vals, cost_diff_2, 0, where=cost_diff_2 < 0, interpolate=True, color='lightcoral', alpha=0.3)

# Zero line
plt.axhline(0, color='gray', linestyle='--', lw=1)

# Labels and styling
plt.xlabel(r'$\mu$ (Drift)', fontsize=12)
plt.ylabel('Cost Difference (Inventory - Bridge)', fontsize=12)
plt.xlim(min_mu, max_mu)
plt.tick_params(axis='both', labelsize=12)
plt.legend(fontsize=12)
plt.tight_layout()

# Show or save
plt.savefig("cost_diff_inventory_vs_bridge_filled.pdf",  bbox_inches='tight', facecolor='auto', edgecolor='auto')
plt.show()
